# Instalar redis no python

In [2]:
!pip install redis

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 7.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install "redis[hiredis]"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


# Rodar redis em um conteiner

Obs: Rodar em um terminal a parte!

In [1]:
!docker login

Authenticating with existing credentials...
Login Succeeded

Logging in with your password grants your terminal complete access to your account. 
For better security, log in with a limited-privilege personal access token. Learn more at https://docs.docker.com/go/access-tokens/


In [2]:
!docker run -p 6379:6379 -it redis/redis-stack:latest

9:C 11 Apr 2023 20:56:25.355 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
9:C 11 Apr 2023 20:56:25.355 # Redis version=6.2.11, bits=64, commit=00000000, modified=0, pid=9, just started
9:C 11 Apr 2023 20:56:25.355 # Configuration loaded
9:M 11 Apr 2023 20:56:25.356 * monotonic clock: POSIX clock_gettime
                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 6.2.11 (00000000/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                  
 (    '      ,       .-`  | `,    )     Running in standalone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 9
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |           https://redis.io       
  `-._    `-._`-.__.-'_.-'    _.-' 

# Execute

## Connect to Redis

In [4]:
import redis
r = redis.Redis(host='localhost', port=6379, db=0)

## Sets

Usa sets, chave de valor único

In [5]:
# Permite inserir chave unica, sobreescrevendo se ja existe
r.set('foo', 'bar')

True

In [6]:
r.get('foo')

b'bar'

In [7]:
r.set('foo', 'bar2')
r.get('foo')

b'bar2'

In [8]:
# Multiplas entradas
data = {
    'hello': 'world',
    'lorem': 'ipsum'
}
r.mset(data)

r.get('lorem') # b'ipsum'

b'ipsum'

In [9]:
r.exists("lorem")

1

In [10]:
# Delete
r.delete('lorem')

1

In [11]:
r.get('lorem')

In [12]:
r.exists("lorem")

0

In [13]:
# Usar json para serializar os objetos como texto
import json
personal_information = {
    'name': 'Rahul',
    'age': 20,
    'address':{
        'house_no': 189,
        'flat_name': 'Golden Flower',
        'area': 'Guindy'
    },
    'languages_known': ['english', 'hindi', 'tamil']
}
r.set('personal_information', json.dumps(personal_information))

True

In [14]:
json.loads(r.get('personal_information'))

{'name': 'Rahul',
 'age': 20,
 'address': {'house_no': 189, 'flat_name': 'Golden Flower', 'area': 'Guindy'},
 'languages_known': ['english', 'hindi', 'tamil']}

### Pipelines

In [15]:
pipe = r.pipeline()
pipe.set('foo', 5)
pipe.set('bar', 18.5)
pipe.set('blee', "hello world!")
pipe.execute()

[True, True, True]

### SADD

In [16]:
r.sadd('ferramentas', 'martelo')
r.sadd('ferramentas', 'serrote')
r.sadd('ferramentas', 'machado')

1

In [17]:
r.sadd('ferramentas', 'martelo')

0

In [18]:
# Mostrar
r.smembers('ferramentas')

{b'machado', b'martelo', b'serrote'}

In [19]:
# Apagar
r.delete('ferramentas')

1

In [20]:
# Mostrar
r.smembers('ferramentas')

set()

In [21]:
# Multiplas
fruits = ["avocado",  "strawberry", "strawberry", "mango", "orange"]
r.sadd('fruits', *fruits)

4

In [22]:
r.smembers('fruits')

{b'avocado', b'mango', b'orange', b'strawberry'}

In [23]:
# Objectos aninhados (nested)
r.hset('person', 'name', 'Ram')
r.hget('person', 'name') # b'Ram'

b'Ram'

### Sets ordenados

In [24]:
r.zadd('test:scores', {'testid:9000': 50})
r.zadd('test:scores', {'testid:8000': 20})
r.zadd('test:scores', {'testid:7000': 45})

1

In [25]:
r.zadd('test:scores', {'testid:7001': 10})

1

In [26]:
# Ordenar
r.zrange('test:scores', 0, -1, withscores=True)

[(b'testid:7001', 10.0),
 (b'testid:8000', 20.0),
 (b'testid:7000', 45.0),
 (b'testid:9000', 50.0)]

In [27]:
r.zadd('test:scores', {'testid:9002': 50, 'b': 2})

2

In [28]:
r.zrange('test:scores', 0, -1, withscores=True)

[(b'b', 2.0),
 (b'testid:7001', 10.0),
 (b'testid:8000', 20.0),
 (b'testid:7000', 45.0),
 (b'testid:9000', 50.0),
 (b'testid:9002', 50.0)]

In [29]:
# Apagar
r.delete('test:scores')

1

In [30]:
r.zrange('test:scores', 0, -1, withscores=True)

[]

## Listas

Cria listas onde a chave pode se repetir

In [31]:
r.rpush('ferramentas', 'martelo')
r.rpush('ferramentas', 'serrote')
r.rpush('ferramentas', 'machado')

3

In [32]:
# Quantidade de entradas para a chave ferramentas
r.llen('ferramentas')

3

In [33]:
# Entrada para ferramentas no indice 1
r.lindex('ferramentas', 3)

In [34]:
# Show list from key
r.lrange('ferramentas', 0, -1)

[b'martelo', b'serrote', b'machado']

In [35]:
# Print the contents of the Redis list and erase
while(r.llen('ferramentas')!=0):
    print(r.rpop('ferramentas'))

b'machado'
b'serrote'
b'martelo'


In [131]:
# Deletar lista
r.delete('ferramentas')

0

In [132]:
# Quantidade de entradas para a chave ferramentas
r.llen('ferramentas')

0

In [133]:
# Guardar uma lista
programming_languages = ['python', 'C#', 'C++', 'C++', 'javascript']
r.lpush('languages', *programming_languages)

5

In [134]:
# Guardar uma lista
programming_languages = ['a', 'b', 'c', 'd', 'e']
r.lpush('letras', *programming_languages)

5

In [135]:
r.lrange('languages', 0, -1)

[b'javascript', b'C++', b'C++', b'C#', b'python']

In [136]:
r.lrange('letras', 0, -1)

[b'e', b'd', b'c', b'b', b'a']

In [137]:
# Deletar lista
r.delete('languages')
r.delete('letras')

1

In [138]:
r.lrange('languages', 0, -1)

[]

In [139]:
r.lrange('letras', 0, -1)

[]

# Queue

In [49]:
print(r.lpush('pedidos', 'calabreza'))
print(r.lpush('pedidos', 'calabreza'))
print(r.lpush('pedidos', 'margherita'))
print(r.lpush('pedidos', 'spaghetti'))
print(r.llen('pedidos'))

1
2
3
4
4


In [50]:
print(r.lpush('compras', 'carne'))
print(r.lpush('compras', 'arroz'))
print(r.lpush('compras', 'frango'))
print(r.llen('compras'))

1
2
3
3


In [51]:
print(r.rpop('pedidos'))
print(r.rpop('pedidos'))
print(r.rpop('pedidos'))
print(r.rpop('pedidos'))
print(r.rpop('pedidos'))
print(r.llen('pedidos'))


b'calabreza'
b'calabreza'
b'margherita'
b'spaghetti'
None
0


In [52]:
print(r.rpop('compras'))
print(r.rpop('compras'))
print(r.rpop('compras'))
print(r.rpop('compras'))
print(r.llen('compras'))

b'carne'
b'arroz'
b'frango'
None
0


## Publish/Subscribe messaging paradigm

In [24]:
# Subscribe
p = r.pubsub()
p.subscribe('my-first-channel', 'my-second-channel')

In [37]:
r.publish('my-first-channel', 'emerson')
r.publish('my-first-channel', 'emerson')
r.publish('my-first-channel', 'testes')

2

In [38]:
r.publish('my-second-channel', 12)
r.publish('my-second-channel', 'fila')

2

In [39]:
while(True):
    message = p.get_message()
    if not message:
        break
    print(message)

{'type': 'message', 'pattern': None, 'channel': b'my-first-channel', 'data': b'emerson'}
{'type': 'message', 'pattern': None, 'channel': b'my-first-channel', 'data': b'emerson'}
{'type': 'message', 'pattern': None, 'channel': b'my-first-channel', 'data': b'testes'}
{'type': 'message', 'pattern': None, 'channel': b'my-second-channel', 'data': b'12'}
{'type': 'message', 'pattern': None, 'channel': b'my-second-channel', 'data': b'fila'}
